In [79]:
import numpy as np

In [80]:
#RMSNorm, girdiyi normalize eder ve stabilite sağlar
# Burada, girdinin elemanlarının karelerinin ortalamasını alıp karekökü ile bölüyoruz.
#

def normalize(x):
    return x/ np.sqrt(np.sum(x**2) + 1e-6) # 1e-8, sıfıra bölmeyi önlemek için eklenir.

In [81]:
normalize(np.array([1,2,3,4,5]))

array([0.13483997, 0.26967994, 0.40451991, 0.53935989, 0.67419986])

In [82]:
# Mixture of Experts
class MixtureOfExperts:
    def __init__(self, num_experts, hidden_dim):
        # Uzmanların (experts) sayısını ve her birinin boyutunu belirtiyoruz.
        self.num_experts = num_experts
        # Her uzman için rastgele ağırlık matrisi oluşturuyoruz.
        self.experts = [np.random.randn(hidden_dim, hidden_dim) for _ in range(num_experts)]

    def route(self, x):
        # Router (yönlendirici), hangi uzmanların kullanılacağını seçer.
        # Basit bir örnek uygulaması: Uzmanlara rastgele skor veriyoruz.
        scores = np.random.rand(self.num_experts)  # Her uzman için rastgele skor oluştur.
        top_k_indices = np.argsort(scores)[-2:]  # En yüksek skora sahip 2 uzmanı seç.
        return top_k_indices

    def forward(self, x):
        # Router tarafından seçilen uzmanlardan geçiş yapıyoruz.
        selected_experts = self.route(x)  # Seçilen uzmanların indekslerini alıyoruz.
        output = 0  # Başlangıç değeri
        for idx in selected_experts:
            # Seçilen uzmanlardan sırasıyla geçiş yapıyoruz.
            output += np.dot(x, self.experts[idx])  # Uzmanın ağırlık matrisi ile çarp.
        return normalize(output)  # Sonucu normalize ederek döndür.

# Örnek kullanım
hidden_dim = 8
x = np.random.randn(1, hidden_dim)  # Tek bir giriş örneği (1 satır, 8 boyut).
moe = MixtureOfExperts(num_experts=4, hidden_dim=hidden_dim)  # 4 uzmanlı sistem oluşturuyoruz.
output = moe.forward(x)  # Girdiyi MoE'den geçiriyoruz.

print("Girdi:\n", x)
print("MoE Çıktısı:\n", output)

Girdi:
 [[ 1.10517535 -1.1437946  -0.6105434   0.61707477 -1.25953267 -1.60542219
  -1.12414634 -0.09881562]]
MoE Çıktısı:
 [[ 0.3041326   0.3533763   0.05777129  0.5359059  -0.3078998  -0.07377402
  -0.33160865  0.53093004]]


In [83]:
class DynamicRouter:
    def __init__(self, num_experts, num_tokens, hidden_size):
        self.num_experts = num_experts
        self.num_tokens = num_tokens
        self.hidden_size = hidden_size
        
        self.weights = np.random.rand(num_experts, num_tokens) # Expertlerin ağırlıkları
        self.bias = np.random.rand(num_experts) # Expertlerin biasları
        
    def route(x):
        scores = np.dot(self.weights, x) + self.bias # Expertlerin skorları
        scores = normalize(scores) # Skorları normalize ediyoruz.
        topk_incdice = np.argsort(scores)[-self.hidden_size:]
        return topk_incdice # En yüksek skorları döndürüyoruz.
    

In [84]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [85]:
# Basit bir veri kümesi (örnek metinler ve etiketler)
data = [
    ("I love this product", 1),
    ("This is amazing", 1),
    ("Absolutely fantastic", 1),
    ("I hate this", 0),
    ("This is terrible", 0),
    ("Absolutely awful", 0),
]

# Metinleri vektörlere dönüştürmek için bir kelime dağarcığı
vocab = {word: idx for idx, word in enumerate(set(" ".join([d[0] for d in data]).split()))}
vocab_size = len(vocab)

# Veriyi sayısal hale getiren bir fonksiyon
def text_to_vector(text, vocab):
    vector = [vocab[word] for word in text.split()]
    return vector

# PyTorch Dataset
from torch.nn.utils.rnn import pad_sequence

class TextDataset(Dataset):
    def __init__(self, data, vocab):
        self.data = [(torch.tensor(text_to_vector(text, vocab)), label) for text, label in data]

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

# Collate fonksiyonu: Dataları aynı uzunluğa getirir.
def collate_fn(batch):
    texts, labels = zip(*batch)
    padded_texts = pad_sequence(texts, batch_first=True, padding_value=0)  # Pad ile aynı uzunluğa getiriliyor.
    labels = torch.tensor(labels)
    return padded_texts, labels

# DataLoader'da collate_fn kullanımı
dataloader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

# Veri kümesi ve DataLoader
dataset = TextDataset(data, vocab)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [86]:
dataset

In [87]:
class MixtureOfExperts(nn.Module):
    def __init__(self, num_experts, embed_dim):
        super().__init__()
        self.num_experts = num_experts
        self.experts = nn.ModuleList([nn.Linear(embed_dim, embed_dim) for _ in range(num_experts)])
        self.gate = nn.Linear(embed_dim, num_experts)

    def forward(self, x):
        # Uzman skorlarını hesapla
        gate_scores = torch.softmax(self.gate(x), dim=-1)  # [batch_size, num_experts]
        
        # Uzmanların çıktıları
        expert_outputs = torch.stack([expert(x) for expert in self.experts], dim=-1)  # [batch_size, embed_dim, num_experts]
        
        # Kapılı uzman karışımı
        output = torch.sum(expert_outputs * gate_scores.unsqueeze(1), dim=-1)  # [batch_size, embed_dim]
        return output

In [88]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_hidden_dim):
        super().__init__()
        self.attention = nn.MultiheadAttention(embed_dim, num_heads)
        self.feed_forward = nn.Sequential(
            nn.Linear(embed_dim, ff_hidden_dim),
            nn.ReLU(),
            nn.Linear(ff_hidden_dim, embed_dim)
        )
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)

    def forward(self, x, padding_mask=None):
        # Self-Attention
        attn_output, _ = self.attention(x, x, x, key_padding_mask=padding_mask)
        x = self.norm1(x + attn_output)
        
        # Feed-Forward
        ff_output = self.feed_forward(x)
        x = self.norm2(x + ff_output)
        return x

In [89]:
def create_padding_mask(input_tensor, padding_value=0):
    # Girdi tensöründe doldurma yapılan yerleri (0 değerini) bulur.
    return (input_tensor == padding_value)

In [90]:
class TransformerWithMoE(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, ff_hidden_dim, num_experts):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.transformer = TransformerBlock(embed_dim, num_heads, ff_hidden_dim)
        self.moe = MixtureOfExperts(num_experts, embed_dim)
        self.classifier = nn.Linear(embed_dim, 2)  # İki sınıf (pozitif ve negatif)

    def forward(self, x):
        # Embedding
        x = self.embedding(x)  # [batch_size, seq_len, embed_dim]
        x = x.permute(1, 0, 2)  # Transformer için [seq_len, batch_size, embed_dim]
        
        # Padding maskesi
        padding_mask = create_padding_mask(x.permute(1, 0, 2)[:, :, 0])  # [batch_size, seq_len]
        
        # Transformer Bloğu
        x = self.transformer(x, padding_mask=padding_mask)
        x = x.permute(1, 0, 2)  # [batch_size, seq_len, embed_dim]
        
        # MoE
        x = self.moe(x.mean(dim=1))  # Sekansın ortalaması alınır (basitlik için)
        
        # Sınıflandırma
        logits = self.classifier(x)
        return logits

In [91]:
class TransformerWithMoE(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_heads, ff_hidden_dim, num_experts):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)  # vocab_size, sözlük boyutuyla eşleşmeli
        self.transformer = TransformerBlock(embed_dim, num_heads, ff_hidden_dim)
        self.moe = MixtureOfExperts(num_experts, embed_dim)
        self.classifier = nn.Linear(embed_dim, 2)  # İki sınıf (pozitif ve negatif)

    def forward(self, x):
        # Embedding
        x = self.embedding(x)  # [batch_size, seq_len, embed_dim]
        x = x.permute(1, 0, 2)  # Transformer için [seq_len, batch_size, embed_dim]
        
        # Transformer Bloğu
        x = self.transformer(x)
        x = x.permute(1, 0, 2)  # [batch_size, seq_len, embed_dim]
        
        # Mixture of Experts (MoE)
        x = self.moe(x.mean(dim=1))  # Sekansın ortalaması alınır (basitlik için)
        
        # Sınıflandırma
        logits = self.classifier(x)
        return logits

In [92]:
import torch
from torch.nn.utils.rnn import pad_sequence

# Hiperparametreler
embed_dim = 16
num_heads = 2
ff_hidden_dim = 32
num_experts = 4
num_epochs = 10
batch_size = 2

# Model, Loss ve Optimizasyon
model = TransformerWithMoE(vocab_size, embed_dim, num_heads, ff_hidden_dim, num_experts)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Custom collate function to pad sequences
def collate_fn(batch):
    inputs, labels = zip(*batch)
    inputs = pad_sequence(inputs, batch_first=True)
    labels = torch.tensor(labels)
    return inputs, labels

# Update your dataloader to use the custom collate function
dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn)

# Eğitim döngüsü
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {total_loss:.4f}")

Epoch 1, Loss: 2.1026
Epoch 2, Loss: 2.0711
Epoch 3, Loss: 2.0545
Epoch 4, Loss: 2.0393
Epoch 5, Loss: 2.0245
Epoch 6, Loss: 2.0096
Epoch 7, Loss: 1.9945
Epoch 8, Loss: 1.9789
Epoch 9, Loss: 1.9626
Epoch 10, Loss: 1.9452


In [93]:
import torch
from torch.nn.utils.rnn import pad_sequence

# Test verileri
test_data = [
    "I absolutely love this",  # Pozitif
    "This is the worst",       # Negatif
    "Fantastic product",       # Pozitif
    "Not good at all",         # Negatif
    "I am very happy",         # Pozitif
    "Completely disappointed"  # Negatif
]

# Metinleri vektörlere dönüştürmek için bir fonksiyon
# Kelime dağarcığı (vocab) oluşturuluyor
vocab = {word: idx for idx, word in enumerate(set(" ".join([d[0] for d in data]).split()))}
vocab['<UNK>'] = len(vocab)  # <UNK> anahtarını sözlüğe ekliyoruz

# Metinleri vektörlere dönüştür ve padding işlemi yap
test_vectors = [torch.tensor(text_to_vector(text, vocab)) for text in test_data]
padded_test_vectors = pad_sequence(test_vectors, batch_first=True, padding_value=0)  # Pad ile aynı uzunlukta yap

print(padded_test_vectors)

KeyError: 'absolutely'

In [ ]:
import torch
from torch.nn.utils.rnn import pad_sequence

# Örnek eğitim verileri (sözlük oluşturmak için)
data = [
    ("I love this product", 1),
    ("This is amazing", 1),
    ("Absolutely fantastic", 1),
    ("I hate this", 0),
    ("This is terrible", 0),
    ("Absolutely awful", 0),
]

# Kelime dağarcığı (vocab) oluşturuluyor
vocab = {word: idx for idx, word in enumerate(set(" ".join([d[0] for d in data]).split()))}
vocab['<UNK>'] = len(vocab)  # <UNK> anahtarını ekliyoruz

# Test verileri
test_data = [
    "I absolutely love this",
    "This is the worst",
    "Fantastic product",
    "Not good at all",
    "I am very happy",
    "Completely disappointed"
]

# Test kelimelerini kontrol et
for text in test_data:
    for word in text.split():
        if word not in vocab:
            print(f"Kelime '{word}' sözlükte yok, '<UNK>' olarak işlenecek.")

# Metinleri vektörlere dönüştürmek için bir fonksiyon
def text_to_vector(text, vocab, unk_token='<UNK>'):
    return [vocab.get(word, vocab[unk_token]) for word in text.split()]

# Metinleri vektörlere dönüştür ve padding işlemi yap
test_vectors = [torch.tensor(text_to_vector(text, vocab)) for text in test_data]

# Sözlük boyutunu kontrol et
vocab_size = len(vocab)

# Tüm indekslerin sınır içinde olup olmadığını kontrol et
for vector in test_vectors:
    if any(idx >= vocab_size for idx in vector):
        print(f"Hata! Bu vektör {vector} sözlük boyutunun dışına çıkıyor.")

# Padding işlemi yap
padded_test_vectors = pad_sequence(test_vectors, batch_first=True, padding_value=0)

# Padding sonrası kontrol et
for vector in padded_test_vectors:
    if any(idx >= vocab_size for idx in vector):
        print(f"Hata! Bu vektör {vector} sözlük boyutunun dışına çıkıyor.")

print("Padded Test Vektörleri:")
print(padded_test_vectors)

# Modeli değerlendirme
model.eval()  # Modeli değerlendirme moduna al
predictions = []

with torch.no_grad():  # Modelin parametrelerini güncelleme
    outputs = model(padded_test_vectors)  # Tüm test girdilerini modele ver
    predicted_classes = torch.argmax(outputs, dim=-1)  # Sınıf tahminlerini al
    predictions = predicted_classes.tolist()  # Liste formatına dönüştür

# Sonuçları yazdır
for text, prediction in zip(test_data, predictions):
    result = "Positive" if prediction == 1 else "Negative"
    print(f"Text: \"{text}\" -> Prediction: {result}")

Kelime 'absolutely' sözlükte yok, '<UNK>' olarak işlenecek.
Kelime 'the' sözlükte yok, '<UNK>' olarak işlenecek.
Kelime 'worst' sözlükte yok, '<UNK>' olarak işlenecek.
Kelime 'Fantastic' sözlükte yok, '<UNK>' olarak işlenecek.
Kelime 'Not' sözlükte yok, '<UNK>' olarak işlenecek.
Kelime 'good' sözlükte yok, '<UNK>' olarak işlenecek.
Kelime 'at' sözlükte yok, '<UNK>' olarak işlenecek.
Kelime 'all' sözlükte yok, '<UNK>' olarak işlenecek.
Kelime 'am' sözlükte yok, '<UNK>' olarak işlenecek.
Kelime 'very' sözlükte yok, '<UNK>' olarak işlenecek.
Kelime 'happy' sözlükte yok, '<UNK>' olarak işlenecek.
Kelime 'Completely' sözlükte yok, '<UNK>' olarak işlenecek.
Kelime 'disappointed' sözlükte yok, '<UNK>' olarak işlenecek.
Padded Test Vektörleri:
tensor([[ 1, 12, 11,  4],
        [ 5,  3, 12, 12],
        [12,  2,  0,  0],
        [12, 12, 12, 12],
        [ 1, 12, 12, 12],
        [12, 12,  0,  0]])


IndexError: index out of range in self